In [2]:
import csv
import ccxt
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# API Key 信息
api_key = "5a62275f-a3b7-4f83-8b0c-d86b978689c4"
api_secret = "BA4CF2390EDC214921684C6A6FB4FB3D"
api_passphrase = "P@ssw0rdNew1234"

# 初始化 OKX 交易所实例
okx = ccxt.okx({
    'apiKey': api_key,
    'secret': api_secret,
    'password': api_passphrase,
    'enableRateLimit': True,  # 避免触发请求限制
})

/Users/rayxu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [92]:
# 配置导出 CSV 文件路径
output_file = 'okx_trade_history.csv'

# 获取账户所有交易对的历史交易记录（过去三个月）
def fetch_trade_history_for_all_pairs():
    # 获取所有交易对
    markets = okx.load_markets()
    symbols = ['BTC/USDT:USDT','ETH/USDT:USDT','BNB/USDT:USDT','SOL/USDT:USDT']
    
    # 计算三个月前的时间戳
    now = datetime.now()
    three_months_ago = now - timedelta(days=100)
    since_timestamp = int(three_months_ago.timestamp() * 1000)  # 转换为毫秒时间戳

    # 写入 CSV 文件的标题行
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Pair", "Trading ID", "Trading Time", "Timestamp(ms)", "Trading Type", "Trading Price", "Trading Amount", "Trading Cost","PnL","Fee"])

    # 遍历每个交易对，获取交易记录
    for symbol in symbols:
        fetch_since = since_timestamp  # 初始化为三个月前的时间戳
        while fetch_since < int(now.timestamp() * 1000):
            try:
                # 获取特定交易对的交易记录，限制单次请求的数量为 100 条
                trades = okx.fetchMyTrades(symbol, since=fetch_since, limit=100, params={'until':(fetch_since+7*3600*24*1000)})
                print(f'Trading Time: {datetime.fromtimestamp(fetch_since/1000)}')
                if not trades:
                    print(f"{symbol} No more Trading Records.")
                    break  # 如果没有返回数据，则跳出循环
                
                # 将每条交易记录写入 CSV 文件
                with open(output_file, mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    for trade in trades:
                        trade_time = datetime.fromtimestamp(trade['timestamp'] / 1000)
                        writer.writerow([
                            symbol,
                            trade['id'],
                            trade_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],  # 精确到毫秒
                            trade['timestamp'],
                            trade['side'],
                            trade['price'],
                            trade['amount'],
                            trade['cost'],
                            trade['info']['fillPnl'],
                            trade['info']['fee']
                        ])
                
                # 更新 `fetch_since` 为最后一笔交易的时间戳，以便获取之后的记录
                fetch_since = int(trades[-1]['timestamp']) + 1  # 加1毫秒避免重复获取
                print(f"{symbol} Successfully fetched the data, continue to fetch data from {datetime.fromtimestamp(fetch_since/1000)}.")

            except Exception as e:
                print(f"Fetching {symbol} error:", e)
                break  # 如果遇到错误则跳出循环

# 执行函数，获取所有交易对的历史交易记录并导出到 CSV 文件
fetch_trade_history_for_all_pairs()

Trading Time: 2024-07-27 17:09:45.471000
BTC/USDT:USDT Successfully fetched the data, continue to fetch data from 2024-08-03 16:20:14.839000.
Trading Time: 2024-08-03 16:20:14.839000
BTC/USDT:USDT Successfully fetched the data, continue to fetch data from 2024-08-09 22:45:15.681000.
Trading Time: 2024-08-09 22:45:15.681000
BTC/USDT:USDT Successfully fetched the data, continue to fetch data from 2024-08-16 20:27:13.480000.
Trading Time: 2024-08-16 20:27:13.480000
BTC/USDT:USDT Successfully fetched the data, continue to fetch data from 2024-08-23 14:55:16.389000.
Trading Time: 2024-08-23 14:55:16.389000
BTC/USDT:USDT Successfully fetched the data, continue to fetch data from 2024-08-30 14:46:12.380000.
Trading Time: 2024-08-30 14:46:12.380000
BTC/USDT:USDT Successfully fetched the data, continue to fetch data from 2024-09-06 12:56:08.723000.
Trading Time: 2024-09-06 12:56:08.723000
BTC/USDT:USDT Successfully fetched the data, continue to fetch data from 2024-09-12 02:25:14.559000.
Tradin

## Data Analysis for OKX Address

This notebook is used to analyze the data of OKX Address.

In [3]:
df = pd.read_csv('okx_trade_history_20240801-20241104.csv')
df.head()

,Pair,Trading ID,Trading Time,Timestamp(ms),Trading Type,Trading Price,Trading Amount,Trading Cost,PnL,Fee
0,BTC/USDT:USDT,958994817,2024-08-01 00:31:47.771,1722443507771,buy,66511.4,3.0,1995.342,0.313154,-0.199534
1,BTC/USDT:USDT,959237852,2024-08-01 03:30:16.181,1722454216181,sell,65771.3,1.0,657.713,0.000000,-0.065771
2,BTC/USDT:USDT,959376531,2024-08-01 04:05:16.088,1722456316088,sell,65372.7,2.0,1307.454,0.000000,-0.130745
3,BTC/USDT:USDT,959463372,2024-08-01 04:35:16.637,1722458116637,sell,64684.9,2.0,1293.698,0.000000,-0.129370
4,BTC/USDT:USDT,959594519,2024-08-01 06:10:15.276,1722463815276,sell,64857.6,1.0,648.576,0.000000,-0.064858


In [4]:
# 将 'Trading Time' 列转换为日期时间格式
df['Trading Time'] = pd.to_datetime(df['Trading Time'])

# 提取日、月份
df['Day'] = df['Trading Time'].dt.to_period('D')
df['Month'] = df['Trading Time'].dt.to_period('M')

# 按日、月份计算 PnL 的总和
Daily_pnl_sum = df.groupby('Day')['PnL'].sum()
monthly_pnl_sum = df.groupby('Month')['PnL'].sum()

Daily_pnl_sum.head()

Day
2024-08-01      22.291127
2024-08-02      32.986876
2024-08-03      72.068880
2024-08-04    1858.750717
2024-08-05    1323.081242
Freq: D, Name: PnL, dtype: float64

In [5]:
monthly_pnl_sum.head()

Month
2024-08    3423.945665
2024-09     354.262184
2024-10     136.539200
2024-11      -7.088700
Freq: M, Name: PnL, dtype: float64